# Notebook: Split Dataset in folds

## Packages

In [7]:
from dataset_statistics import df_statistics
import json

## Parameters

In [8]:
DATASET_PATH = "dataset_total/total_dataset.json"

## Code

### Create Statistics

In [9]:
with open(DATASET_PATH, 'r', encoding='utf-8') as json_file:
    dataset = json.load(json_file)

df_statistics(dataset, False)

{'most_frequent_aspect_terms_FOOD': [('Essen', 258),
  ('Bier', 39),
  ('Speisen', 32),
  ('Fleisch', 28),
  ('Küche', 25),
  ('Gerichte', 20),
  ('Bratkartoffeln', 19),
  ('Portionen', 17),
  ('Schnitzel', 17),
  ('Tapas', 13)],
 'unique_aspect_terms_FOOD': 520,
 'number_of_implicit_aspects_FOOD': 152,
 'number_of_explicit_aspects_FOOD': 1282,
 'number_of_aspects_FOOD': 1434,
 'most_frequent_aspect_terms_SERVICE': [('Service', 169),
  ('Bedienung', 106),
  ('Personal', 75),
  ('Kellner', 47),
  ('Kellnerin', 13),
  ('Bedienungen', 8),
  ('Mitarbeiter', 7),
  ('Empfang', 7),
  ('Reservierung', 7),
  ('Servicepersonal', 6)],
 'unique_aspect_terms_SERVICE': 169,
 'number_of_implicit_aspects_SERVICE': 181,
 'number_of_explicit_aspects_SERVICE': 674,
 'number_of_aspects_SERVICE': 855,
 'most_frequent_aspect_terms_PRICE': [('Preise', 26),
  ('Preis', 20),
  ('Preisen', 10),
  ('Essen', 9),
  ('Preis-Leistungsverhältnis', 8),
  ('Getränke', 5),
  ('Wein', 5),
  ('Geld', 5),
  ('Preis Leistun